# Hypothesis 3: Hallucination Reduction and Time-to-Market Impact
**Implementing hallucination-reducing techniques in LLMs significantly improves (>30%) time to market in new product development.**

In [ ]:
# Cell [0] - Setup and Imports
# Purpose: Import all required libraries and configure environment settings for SiFP COSMIC Estimation Analysis
# Dependencies: pandas, numpy, matplotlib, seaborn, scipy, neo4j, scikit-learn, dotenv
# Breadcrumbs: Setup -> Environment Configuration -> Analysis Preparation

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from neo4j import GraphDatabase
from dotenv import load_dotenv
import json
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

def setup_analysis_environment():
    """
    Configure analysis environment with display options and styling
    
    Returns:
        dict: Configuration parameters for the analysis
    """
    # Suppress warnings for cleaner output
    warnings.filterwarnings('ignore')
    
    # Configure matplotlib and seaborn styling
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # Configure pandas display options for better readability
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    
    # Load environment variables
    load_dotenv()
    
    # Configuration parameters
    config = {
        'NEO4J_URI': os.getenv('NEO4J_URI'),
        'NEO4J_USER': os.getenv('NEO4J_USER'),
        'NEO4J_PASSWORD': os.getenv('NEO4J_PASSWORD'),
        'NEO4J_PROJECT_NAME': os.getenv('NEO4J_PROJECT_NAME'),
        'CONVERSION_FACTOR': 0.957,  # SiFP = 0.957 × UFP (Desharnais)
        'COST_PER_HOUR': 100  # Industry standard for cost impact calculations
    }
    
    print("✓ Analysis environment configured successfully")
    print(f"✓ Project: {config['NEO4J_PROJECT_NAME']}")
    
    return config

# Execute setup when cell runs
CONFIG = setup_analysis_environment()

In [ ]:
# Cell [1] - Load and Process Java Code Metrics
# Purpose: Load actual implementation metrics from iTrust java.csv for baseline establishment
# Dependencies: pandas, configured environment (Cell 0)
# Breadcrumbs: Setup -> Code Metrics Loading -> Baseline Data Preparation

def load_code_metrics():
    """
    Load and process Java code metrics from iTrust dataset
    
    Returns:
        pd.DataFrame: Processed code metrics with derived calculations
    """
    try:
        # Load java.csv from iTrust dataset
        java_df = pd.read_csv('../datasets/iTrust/iTrust/java.csv')
        print(f"✓ Loaded java.csv: {java_df.shape[0]} code entities")

        # Filter to only File entries (excluding methods, functions, etc.)
        file_df = java_df[java_df['Kind'] == 'File'].copy()
        print(f"  Files: {len(file_df)}")

        # Select relevant metrics for SiFP analysis
        metrics_columns = [
            'Name', 'CountLine', 'CountLineCode', 'CountLineComment',
            'CountDeclClass', 'CountDeclMethod', 'CountDeclMethodAll',
            'CountDeclExecutableUnit', 'Cyclomatic', 'MaxCyclomatic'
        ]

        # Create analysis-ready dataframe
        code_metrics_df = file_df[metrics_columns].copy()

        # Calculate derived metrics that correlate with function points
        code_metrics_df['TotalUnits'] = (
            code_metrics_df['CountDeclClass'] + 
            code_metrics_df['CountDeclMethod']
        )

        return code_metrics_df
        
    except Exception as e:
        print(f"Error loading code metrics: {e}")
        raise

# Load and process the code metrics
code_metrics_df = load_code_metrics()

# Display sample data for verification
print("\nSample code metrics:")
print(code_metrics_df.head())

# Calculate and display summary statistics
print("\nCode Metrics Summary:")
print("=" * 40)
print(f"  Total files: {len(code_metrics_df)}")
print(f"  Average lines of code: {code_metrics_df['CountLineCode'].mean():.0f}")
print(f"  Average methods per file: {code_metrics_df['CountDeclMethod'].mean():.1f}")
print(f"  Average cyclomatic complexity: {code_metrics_df['Cyclomatic'].mean():.1f}")
print(f"  Total LOC in codebase: {code_metrics_df['CountLineCode'].sum():,}")

# Store key metrics for later analysis
code_summary = {
    'total_files': len(code_metrics_df),
    'total_lines': code_metrics_df['CountLineCode'].sum(),
    'total_classes': code_metrics_df['CountDeclClass'].sum(),
    'total_methods': code_metrics_df['CountDeclMethod'].sum(),
    'avg_complexity': code_metrics_df['Cyclomatic'].mean(),
    'total_units': code_metrics_df['TotalUnits'].sum()
}

print(f"\n✓ Code metrics loaded and processed successfully")

In [ ]:
# Cell [2] - Connect to Neo4j and Retrieve LLM SiFP Estimates
# Purpose: Retrieve LLM-generated SiFP estimates for requirements with established ground truth links
# Dependencies: Neo4j connection, json processing, CONFIG from Cell 0
# Breadcrumbs: Setup -> Code Metrics -> Neo4j Data Retrieval -> LLM Estimates Analysis

def retrieve_llm_estimates():
    """
    Connect to Neo4j and retrieve LLM SiFP estimates for ground truth requirements
    
    Returns:
        tuple: (llm_estimates_df, ground_truth_requirements, model_success_df)
    """
    try:
        # Establish Neo4j connection using configuration
        driver = GraphDatabase.driver(
            CONFIG['NEO4J_URI'], 
            auth=(CONFIG['NEO4J_USER'], CONFIG['NEO4J_PASSWORD'])
        )
        print(f"✓ Connected to Neo4j for project: {CONFIG['NEO4J_PROJECT_NAME']}")

        with driver.session() as session:
            # First, identify TARGET requirements with ground truth
            gt_query = """
                MATCH (r1:Requirement {project: $project_name, type: 'TARGET'})
                WHERE EXISTS((r1)-[:GROUND_TRUTH]-()) OR EXISTS(()-[:GROUND_TRUTH]-(r1))
                RETURN DISTINCT r1.id as requirement_id
            """
            
            gt_result = session.run(gt_query, project_name=CONFIG['NEO4J_PROJECT_NAME'])
            ground_truth_requirements = [record['requirement_id'] for record in gt_result]
            
            print(f"✓ Found {len(ground_truth_requirements)} TARGET requirements with ground truth")

            # Query for LLM estimates on ground truth requirements
            estimation_query = """
                MATCH (r1:Requirement {project: $project_name, type: 'TARGET'})-[se:SIFP_ESTIMATION]->(r2:Requirement)
                WHERE r1.id IN $ground_truth_reqs
                  AND se.final_estimation IS NOT NULL
                  AND se.is_valid = true
                RETURN DISTINCT r1.id as requirement_id,
                       r1.content as requirement_content,
                       se.model as model,
                       se.actor_analysis as actor_json,
                       se.final_estimation as final_json,
                       se.judge_evaluation as judge_eval_json,
                       se.confidence as confidence,
                       se.judge_confidence as judge_confidence,
                       se.judge_score as judge_score
                ORDER BY r1.id, se.model
            """
            
            result = session.run(estimation_query, 
                               project_name=CONFIG['NEO4J_PROJECT_NAME'], 
                               ground_truth_reqs=ground_truth_requirements)
            
            # Process and structure the results
            records = []
            for record in result:
                try:
                    # Parse JSON data from Neo4j
                    actor_data = json.loads(record['actor_json']) if record['actor_json'] else {}
                    final_data = json.loads(record['final_json']) if record['final_json'] else {}
                    
                    # Extract UGEP and UGDG counts
                    actor_ugep = len(actor_data.get('ugeps', []))
                    actor_ugdg = len(actor_data.get('ugdgs', []))
                    final_ugep = len(final_data.get('ugeps', []))
                    final_ugdg = len(final_data.get('ugdgs', []))
                    
                    # Calculate SiFP using standard formula: SiFP = 4.6 × UGEP + 7.0 × UGDG
                    actor_sifp = 4.6 * actor_ugep + 7 * actor_ugdg
                    final_sifp = 4.6 * final_ugep + 7 * final_ugdg
                    
                    records.append({
                        'requirement_id': record['requirement_id'],
                        'requirement_content': record['requirement_content'][:100] + '...',
                        'model': record['model'],
                        'actor_ugep': actor_ugep,
                        'actor_ugdg': actor_ugdg,
                        'actor_sifp': actor_sifp,
                        'final_ugep': final_ugep,
                        'final_ugdg': final_ugdg,
                        'final_sifp': final_sifp,
                        'judge_score': record['judge_score'],
                        'confidence': record['confidence']
                    })
                except Exception as e:
                    print(f"Warning: Error processing record for {record.get('requirement_id', 'unknown')}: {e}")
                    continue

        # Create estimates DataFrame
        llm_estimates_df = pd.DataFrame(records)
        
        if not llm_estimates_df.empty:
            print(f"\n✓ Retrieved {len(llm_estimates_df)} LLM estimates for ground truth requirements")
            
            # Calculate model success rates
            print("\nModel Success Rates (Ground Truth Requirements):")
            print("=" * 50)
            
            model_success = []
            for model in sorted(llm_estimates_df['model'].unique()):
                model_estimates = llm_estimates_df[llm_estimates_df['model'] == model]
                successful_reqs = model_estimates['requirement_id'].nunique()
                success_rate = successful_reqs / len(ground_truth_requirements) * 100
                
                model_success.append({
                    'model': model,
                    'successful_estimates': successful_reqs,
                    'total_ground_truth': len(ground_truth_requirements),
                    'success_rate': success_rate
                })
                
                print(f"  {model}: {successful_reqs}/{len(ground_truth_requirements)} ({success_rate:.1f}%)")
            
            model_success_df = pd.DataFrame(model_success)
            
            # Display sample estimates for verification
            print("\nSample LLM estimates (with ground truth):")
            display_cols = ['requirement_id', 'model', 'final_sifp', 'judge_score']
            print(llm_estimates_df[display_cols].head())
            
            return llm_estimates_df, ground_truth_requirements, model_success_df
            
        else:
            print("Warning: No LLM estimates found for ground truth requirements!")
            return pd.DataFrame(), ground_truth_requirements, pd.DataFrame()
            
    except Exception as e:
        print(f"Error retrieving LLM estimates: {e}")
        raise
    finally:
        driver.close()
        print("✓ Neo4j connection closed")

# Execute the retrieval process
llm_estimates_df, ground_truth_requirements, model_success_df = retrieve_llm_estimates()

In [ ]:
# Cell [3] - Establish Requirements-to-Code Mapping and Feature Analysis
# Purpose: Create mapping between requirements and actual code files for validation baseline
# Dependencies: llm_estimates_df from Cell 2, feature extraction logic
# Breadcrumbs: Setup -> Data Retrieval -> Requirements Mapping -> Feature Analysis

def analyze_requirement_features():
    """
    Analyze requirements by extracting feature identifiers and establishing mappings
    
    Returns:
        tuple: (feature_requirements_df, feature_mapping)
    """
    
    def extract_feature_from_requirement(req_id):
        """
        Extract feature/module name from requirement ID using common patterns
        
        Args:
            req_id (str): Requirement identifier
            
        Returns:
            str: Extracted feature name
        """
        # Handle common requirement ID patterns
        if 'UC' in req_id:
            # Use case format: UC1.1 -> UC1
            return req_id.split('.')[0]
        elif '-' in req_id:
            # Functional requirement format: FR-AUTH-001 -> AUTH
            parts = req_id.split('-')
            if len(parts) >= 2:
                return parts[1]
        return req_id  # Return original if no pattern matches

    # Check if we have LLM estimates to analyze
    if not llm_estimates_df.empty:
        print("Analyzing requirement features and groupings...")
        print("=" * 50)
        
        # Extract features from requirement IDs
        llm_estimates_df['feature'] = llm_estimates_df['requirement_id'].apply(extract_feature_from_requirement)
        
        # Group requirements by feature for analysis
        feature_requirements = llm_estimates_df.groupby('feature').agg({
            'requirement_id': 'nunique',  # Count unique requirements
            'final_sifp': ['mean', 'sum', 'std'],
            'model': 'nunique'  # Count how many models estimated this feature
        }).round(2)
        
        # Flatten column names for better readability
        feature_requirements.columns = [
            'unique_requirements', 'avg_sifp', 'total_sifp', 'std_sifp', 'models_count'
        ]
        
        # Sort by total SiFP for better insights
        feature_requirements = feature_requirements.sort_values('total_sifp', ascending=False)
        
        print("Requirements grouped by feature:")
        print(feature_requirements)
        
        # Calculate feature statistics
        print(f"\nFeature Analysis Summary:")
        print(f"  Total features identified: {len(feature_requirements)}")
        print(f"  Average requirements per feature: {feature_requirements['unique_requirements'].mean():.1f}")
        print(f"  Average SiFP per feature: {feature_requirements['avg_sifp'].mean():.1f}")
        print(f"  Most complex feature: {feature_requirements.index[0]} ({feature_requirements['total_sifp'].max():.1f} SiFP)")
        
        # Create feature mapping for traceability
        feature_mapping = {}
        for feature in feature_requirements.index:
            feature_reqs = llm_estimates_df[llm_estimates_df['feature'] == feature]['requirement_id'].unique()
            feature_mapping[feature] = {
                'requirements': list(feature_reqs),
                'count': len(feature_reqs),
                'estimated_loc': feature_requirements.loc[feature, 'total_sifp'] * CONFIG.get('avg_loc_per_sifp', 100)
            }
        
        return feature_requirements, feature_mapping
        
    else:
        print("Warning: No LLM estimates available for feature analysis")
        return pd.DataFrame(), {}

# Execute feature analysis
if not llm_estimates_df.empty:
    feature_requirements_df, feature_mapping = analyze_requirement_features()
    
    # Display insights about the mapping approach
    print(f"\nRequirement-to-Code Mapping Approach:")
    print("=" * 50)
    print("• Using aggregate analysis based on requirement feature groupings")
    print("• Features extracted from requirement IDs using pattern matching")
    print("• In production, explicit traceability links would provide direct mapping")
    print("• Current approach enables statistical validation at feature level")
    
else:
    print("Skipping feature analysis - no LLM estimates available")
    feature_requirements_df = pd.DataFrame()
    feature_mapping = {}

In [ ]:
# Cell [4] - Calculate Normalized Metrics and Establish Conversion Baselines
# Purpose: Establish normalized relationships between SiFP and code metrics using UFP→SiFP conversion
# Dependencies: code_summary from Cell 1, llm_estimates_df from Cell 2, CONFIG from Cell 0
# Breadcrumbs: Setup -> Data Collection -> Mapping -> Baseline Establishment

def calculate_normalized_metrics():
    """
    Calculate normalized metrics and establish baseline relationships between SiFP and code metrics
    
    Returns:
        tuple: (llm_analysis_df, baseline_metrics, industry_metrics)
    """
    
    print("Code Base Summary (Full Codebase):")
    print("=" * 40)
    for key, value in code_summary.items():
        print(f"  {key}: {value:.0f}")

    # Calculate normalized code metrics
    print("\nNormalized Code Metrics (Full Codebase):")
    print("-" * 40)

    # Key normalized metrics
    loc_per_file = code_summary['total_lines'] / code_summary['total_files']
    methods_per_kloc = (code_summary['total_methods'] / code_summary['total_lines']) * 1000
    classes_per_kloc = (code_summary['total_classes'] / code_summary['total_lines']) * 1000

    print(f"  Lines of code per file: {loc_per_file:.1f}")
    print(f"  Methods per KLOC: {methods_per_kloc:.1f}")
    print(f"  Classes per KLOC: {classes_per_kloc:.1f}")

    # Establish industry baselines
    print("\n" + "="*60)
    print("BASELINE CALCULATION APPROACHES")
    print("="*60)

    # Industry standards from research
    INDUSTRY_LOC_PER_UFP = 100  # Typical for Java
    INDUSTRY_LOC_PER_SIFP = INDUSTRY_LOC_PER_UFP / CONFIG['CONVERSION_FACTOR']  # Adjust for conversion

    industry_metrics = {
        'LOC_PER_UFP': INDUSTRY_LOC_PER_UFP,
        'LOC_PER_SIFP': INDUSTRY_LOC_PER_SIFP,
        'SIFP_PER_KLOC': 1000/INDUSTRY_LOC_PER_SIFP
    }

    print(f"\nIndustry Baseline (Research-based):")
    print(f"  Typical LOC per UFP (Java): {INDUSTRY_LOC_PER_UFP}")
    print(f"  Implied LOC per SiFP: {INDUSTRY_LOC_PER_SIFP:.1f}")
    print(f"  SiFP per KLOC: {industry_metrics['SIFP_PER_KLOC']:.2f}")

    # Analyze LLM estimates if available
    if not llm_estimates_df.empty and len(ground_truth_requirements) > 0:
        print("\n" + "="*60)
        print("LLM SIFP ANALYSIS (Scaled to Estimated Requirements)")
        print("="*60)
        
        # Get unique requirements that were successfully estimated
        estimated_requirements = llm_estimates_df['requirement_id'].unique()
        estimation_coverage = len(estimated_requirements) / len(ground_truth_requirements)
        
        print(f"\nEstimation Coverage:")
        print(f"  Ground truth requirements: {len(ground_truth_requirements)}")
        print(f"  Requirements with estimates: {len(estimated_requirements)} ({estimation_coverage:.1%})")
        
        # Scale code metrics based on estimation coverage
        scaled_code_metrics = {
            'lines': code_summary['total_lines'] * estimation_coverage,
            'classes': code_summary['total_classes'] * estimation_coverage,
            'methods': code_summary['total_methods'] * estimation_coverage
        }
        
        print(f"\nScaled Code Metrics (for estimated requirements only):")
        print(f"  Estimated lines of code: {scaled_code_metrics['lines']:.0f}")
        print(f"  Estimated classes: {scaled_code_metrics['classes']:.0f}")
        print(f"  Estimated methods: {scaled_code_metrics['methods']:.0f}")
        
        # Calculate metrics for each LLM model
        llm_analysis = []
        
        for model in sorted(llm_estimates_df['model'].unique()):
            model_data = llm_estimates_df[llm_estimates_df['model'] == model]
            
            # Calculate model totals
            total_sifp = model_data['final_sifp'].sum()
            total_ugep = model_data['final_ugep'].sum()
            total_ugdg = model_data['final_ugdg'].sum()
            successful_reqs = model_data['requirement_id'].nunique()
            
            # Calculate normalized metrics based on MODEL-SPECIFIC coverage
            model_coverage = successful_reqs / len(ground_truth_requirements)
            model_estimated_loc = code_summary['total_lines'] * model_coverage
            
            # Key normalized metrics
            sifp_per_kloc = (total_sifp / model_estimated_loc) * 1000 if model_estimated_loc > 0 else 0
            loc_per_sifp = model_estimated_loc / total_sifp if total_sifp > 0 else 0
            sifp_per_req = total_sifp / successful_reqs if successful_reqs > 0 else 0
            
            # Calculate equivalent UFP for comparison
            equivalent_ufp = total_sifp / CONFIG['CONVERSION_FACTOR']
            
            llm_analysis.append({
                'model': model,
                'successful_reqs': successful_reqs,
                'coverage': model_coverage,
                'total_sifp': total_sifp,
                'equivalent_ufp': equivalent_ufp,
                'total_ugep': total_ugep,
                'total_ugdg': total_ugdg,
                'estimated_loc': model_estimated_loc,
                'sifp_per_kloc': sifp_per_kloc,
                'loc_per_sifp': loc_per_sifp,
                'sifp_per_req': sifp_per_req
            })
            
            print(f"\n{model}:")
            print(f"  Successfully estimated: {successful_reqs}/{len(ground_truth_requirements)} requirements ({model_coverage:.1%})")
            print(f"  Total SiFP: {total_sifp:.1f} (equivalent to {equivalent_ufp:.1f} UFP)")
            print(f"  Estimated LOC coverage: {model_estimated_loc:.0f} lines")
            print(f"  SiFP per KLOC: {sifp_per_kloc:.2f}")
            print(f"  LOC per SiFP point: {loc_per_sifp:.1f}")
            print(f"  Deviation from industry baseline: {(loc_per_sifp - INDUSTRY_LOC_PER_SIFP)/INDUSTRY_LOC_PER_SIFP*100:+.1f}%")
        
        # Create analysis DataFrame
        llm_analysis_df = pd.DataFrame(llm_analysis)
        
        # Calculate project-specific baseline (weighted average of LLM estimates)
        if not llm_analysis_df.empty:
            weighted_loc_per_sifp = np.average(llm_analysis_df['loc_per_sifp'], 
                                              weights=llm_analysis_df['coverage'])
            
            baseline_metrics = {
                'project_loc_per_sifp': weighted_loc_per_sifp,
                'industry_loc_per_sifp': INDUSTRY_LOC_PER_SIFP,
                'difference_pct': (weighted_loc_per_sifp - INDUSTRY_LOC_PER_SIFP)/INDUSTRY_LOC_PER_SIFP*100,
                'estimated_requirements': estimated_requirements,
                'scaled_code_metrics': scaled_code_metrics
            }
            
            print("\n\nBASELINE COMPARISON:")
            print("-" * 40)
            print(f"  Industry baseline LOC/SiFP: {INDUSTRY_LOC_PER_SIFP:.1f}")
            print(f"  Project baseline LOC/SiFP (weighted avg): {weighted_loc_per_sifp:.1f}")
            print(f"  Difference: {baseline_metrics['difference_pct']:+.1f}%")
            
            return llm_analysis_df, baseline_metrics, industry_metrics
        
    # Return empty results if no LLM data
    return pd.DataFrame(), {}, industry_metrics

# Execute the normalized metrics calculation
llm_analysis_df, baseline_metrics, industry_metrics = calculate_normalized_metrics()

print(f"\n✓ Normalized metrics calculated successfully")

In [ ]:
# Cell [5] - Detailed Normalized Performance Analysis
# Purpose: Analyze model accuracy in normalized units (per SiFP point) with quality metrics
# Dependencies: llm_analysis_df and baseline_metrics from Cell 4, llm_estimates_df from Cell 2  
# Breadcrumbs: Setup -> Data Collection -> Baseline Establishment -> Performance Analysis

def analyze_model_performance():
    """
    Analyze detailed performance metrics for each LLM model
    
    Returns:
        pd.DataFrame: Performance analysis with accuracy rankings
    """
    
    if llm_estimates_df.empty or llm_analysis_df.empty:
        print("Warning: No LLM data available for performance analysis")
        return pd.DataFrame()
    
    print("Normalized Model Performance Analysis (Per SiFP Point)")
    print("=" * 60)
    
    model_performance = []
    
    # Use the project baseline from calculated metrics
    baseline_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', 
                                                industry_metrics.get('LOC_PER_SIFP', 100))
    
    print(f"Using baseline: {baseline_loc_per_sifp:.1f} LOC per SiFP")
    print("-" * 60)
    
    for _, row in llm_analysis_df.iterrows():
        model = row['model']
        
        # Calculate normalized accuracy metrics
        loc_per_sifp_error = row['loc_per_sifp'] - baseline_loc_per_sifp
        loc_per_sifp_error_pct = (loc_per_sifp_error / baseline_loc_per_sifp) * 100 if baseline_loc_per_sifp > 0 else 0
        
        # Get quality metrics from original LLM data
        model_data = llm_estimates_df[llm_estimates_df['model'] == model]
        avg_confidence = model_data['confidence'].mean() if 'confidence' in model_data.columns else 0
        avg_judge_score = model_data['judge_score'].mean() if 'judge_score' in model_data.columns else 0
        std_sifp = model_data['final_sifp'].std() if 'final_sifp' in model_data.columns else 0
        
        # Calculate success rate
        success_rate = row['successful_reqs'] / len(ground_truth_requirements) if len(ground_truth_requirements) > 0 else 0
        
        model_performance.append({
            'Model': model,
            'Success_Rate': success_rate,
            'SiFP_per_KLOC': row['sifp_per_kloc'],
            'LOC_per_SiFP': row['loc_per_sifp'],
            'LOC_per_SiFP_Error': loc_per_sifp_error,
            'Error_Pct': abs(loc_per_sifp_error_pct),
            'Avg_SiFP_per_Req': row['sifp_per_req'],
            'Std_SiFP': std_sifp,
            'Avg_Confidence': avg_confidence,
            'Avg_Judge_Score': avg_judge_score
        })
        
        # Display individual model analysis
        print(f"\n{model}:")
        print(f"  Success rate: {success_rate:.1%}")
        print(f"  SiFP per KLOC: {row['sifp_per_kloc']:.2f}")
        print(f"  LOC per SiFP point: {row['loc_per_sifp']:.1f}")
        print(f"  Error vs baseline: {loc_per_sifp_error:+.1f} LOC/SiFP ({loc_per_sifp_error_pct:+.1f}%)")
        print(f"  Average confidence: {avg_confidence:.2%}")
        print(f"  Average judge score: {avg_judge_score:.2f}/5")
        print(f"  SiFP variability (std): {std_sifp:.2f}")
    
    # Create performance DataFrame
    performance_df = pd.DataFrame(model_performance)
    
    if not performance_df.empty:
        # Rank models by normalized accuracy (lower error is better)
        performance_df['Accuracy_Rank'] = performance_df['Error_Pct'].rank()
        
        print("\n\nNormalized Performance Summary:")
        print("=" * 60)
        summary_cols = ['Model', 'Success_Rate', 'LOC_per_SiFP', 'Error_Pct', 'Accuracy_Rank']
        print(performance_df[summary_cols].round(3).to_string(index=False))
        
        # Additional insights
        best_accuracy = performance_df.loc[performance_df['Error_Pct'].idxmin()]
        best_coverage = performance_df.loc[performance_df['Success_Rate'].idxmax()]
        
        print(f"\nKey Insights:")
        print(f"  Most accurate model: {best_accuracy['Model']} ({best_accuracy['Error_Pct']:.1f}% error)")
        print(f"  Best coverage model: {best_coverage['Model']} ({best_coverage['Success_Rate']:.1%} success rate)")
        print(f"  Average error across all models: {performance_df['Error_Pct'].mean():.1f}%")
        
    return performance_df

# Execute performance analysis
performance_df = analyze_model_performance()

print(f"\n✓ Performance analysis completed successfully")

In [ ]:
# Cell [6] - Load Desharnais Dataset and Establish UFP→SiFP→Effort Relationships  
# Purpose: Load industry benchmark dataset and establish the complete conversion chain for effort estimation
# Dependencies: sklearn LinearRegression, CONFIG from Cell 0, pandas processing
# Breadcrumbs: Setup -> Performance Analysis -> Industry Benchmarks -> Effort Conversion Chain

def load_and_analyze_desharnais():
    """
    Load Desharnais dataset and establish UFP→SiFP→Effort conversion relationships
    
    Returns:
        tuple: (desharnais_df, effort_metrics, effort_model)
    """
    try:
        # Load industry benchmark dataset
        desharnais_df = pd.read_csv('../datasets/CostEstimation/Desharnais.csv')
        print(f"✓ Loaded Desharnais dataset: {desharnais_df.shape[0]} projects")

        # Identify column names (handle variations in dataset)
        ufp_column = 'PointsNonAdjust' if 'PointsNonAdjust' in desharnais_df.columns else 'UFP'
        effort_column = 'Effort' if 'Effort' in desharnais_df.columns else 'effort'

        print(f"Using columns: UFP='{ufp_column}', Effort='{effort_column}'")
        
        # Apply UFP to SiFP conversion using research-validated factor
        print(f"\nApplying UFP→SiFP conversion factor: {CONFIG['CONVERSION_FACTOR']}")
        desharnais_df['SiFP_converted'] = desharnais_df[ufp_column] * CONFIG['CONVERSION_FACTOR']

        # Calculate effort per SiFP metrics
        print("\nDesharnais Normalized Metrics:")
        print("=" * 40)

        # Calculate hours per SiFP point for each project
        desharnais_df['hours_per_sifp'] = desharnais_df[effort_column] / desharnais_df['SiFP_converted']

        # Calculate summary statistics
        effort_metrics = {
            'avg_hours_per_sifp': desharnais_df['hours_per_sifp'].mean(),
            'median_hours_per_sifp': desharnais_df['hours_per_sifp'].median(),
            'std_hours_per_sifp': desharnais_df['hours_per_sifp'].std(),
            'min_hours_per_sifp': desharnais_df['hours_per_sifp'].min(),
            'max_hours_per_sifp': desharnais_df['hours_per_sifp'].max()
        }

        print(f"  Average hours per SiFP: {effort_metrics['avg_hours_per_sifp']:.2f}")
        print(f"  Median hours per SiFP: {effort_metrics['median_hours_per_sifp']:.2f}")
        print(f"  Std dev hours per SiFP: {effort_metrics['std_hours_per_sifp']:.2f}")
        print(f"  Range: {effort_metrics['min_hours_per_sifp']:.2f} - {effort_metrics['max_hours_per_sifp']:.2f}")

        # Build linear effort prediction model
        print(f"\nBuilding Linear Effort Model:")
        print("-" * 30)
        
        # Prepare data for sklearn
        X = desharnais_df[['SiFP_converted']].values.astype(np.float64)
        y = desharnais_df[effort_column].values.astype(np.float64)

        # Fit linear regression model
        effort_model = LinearRegression()
        effort_model.fit(X, y)

        # Extract model coefficients
        linear_hours_per_sifp = float(effort_model.coef_[0])
        intercept = float(effort_model.intercept_)
        
        # Calculate model performance
        y_pred = effort_model.predict(X)
        r2 = float(r2_score(y, y_pred))

        print(f"  Hours per SiFP (coefficient): {linear_hours_per_sifp:.2f}")
        print(f"  Base hours (intercept): {intercept:.2f}")
        print(f"  R² score: {r2:.3f}")
        
        # Add model metrics to effort_metrics
        effort_metrics.update({
            'linear_hours_per_sifp': linear_hours_per_sifp,
            'intercept': intercept,
            'r2_score': r2
        })

        # Analyze SiFP distribution in industry data
        print(f"\nSiFP Distribution in Desharnais Dataset:")
        print("-" * 40)
        print(f"  Mean SiFP per project: {desharnais_df['SiFP_converted'].mean():.1f}")
        print(f"  Median SiFP per project: {desharnais_df['SiFP_converted'].median():.1f}")
        print(f"  Range: {desharnais_df['SiFP_converted'].min():.1f} - {desharnais_df['SiFP_converted'].max():.1f}")
        print(f"  Total projects: {len(desharnais_df)}")

        return desharnais_df, effort_metrics, effort_model
        
    except Exception as e:
        print(f"Error loading Desharnais dataset: {e}")
        raise

# Execute Desharnais analysis
desharnais_df, effort_metrics, effort_model = load_and_analyze_desharnais()

print(f"\n✓ Desharnais dataset analysis completed successfully")

In [ ]:
# Cell [7] - Normalized Effort Impact Analysis with Complete Conversion Chain
# Purpose: Analyze effort impact using UFP→SiFP→Effort conversion chain and calculate cost implications  
# Dependencies: performance_df from Cell 5, effort_metrics from Cell 6, CONFIG from Cell 0
# Breadcrumbs: Setup -> Performance Analysis -> Industry Benchmarks -> Effort Impact Analysis

def analyze_effort_impact():
    """
    Analyze effort impact using the complete UFP→SiFP→Effort conversion chain
    
    Returns:
        pd.DataFrame: Effort impact analysis with cost implications
    """
    
    if performance_df.empty or not effort_metrics:
        print("Warning: Missing required data for effort impact analysis")
        return pd.DataFrame()
    
    print("Normalized Effort Impact Analysis (Per SiFP Point)")
    print("=" * 60)

    # Display conversion chain information
    print(f"\nConversion Chain:")
    print(f"  UFP → SiFP: Factor = {CONFIG['CONVERSION_FACTOR']} (SiFP = {CONFIG['CONVERSION_FACTOR']} × UFP)")
    print(f"  SiFP → Effort: {effort_metrics['avg_hours_per_sifp']:.2f} hours/SiFP (from Desharnais)")
    
    if baseline_metrics:
        print(f"  SiFP → LOC: {baseline_metrics['project_loc_per_sifp']:.1f} LOC/SiFP (project baseline)")
    
    effort_impact = []
    
    for _, row in performance_df.iterrows():
        model = row['Model']
        
        # Get model's LOC per SiFP
        model_loc_per_sifp = row['LOC_per_SiFP']
        baseline_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', 
                                                   industry_metrics.get('LOC_PER_SIFP', 100))
        
        # Calculate SiFP estimation accuracy
        # If model estimates fewer LOC per SiFP, it's overestimating SiFP count
        sifp_estimation_factor = baseline_loc_per_sifp / model_loc_per_sifp if model_loc_per_sifp > 0 else 1
        
        # Calculate effort impact using Desharnais baseline
        baseline_hours_per_sifp = effort_metrics['avg_hours_per_sifp']
        
        # The effective hours per estimated SiFP
        effective_hours_per_estimated_sifp = baseline_hours_per_sifp / sifp_estimation_factor
        
        # Calculate percentage error in effort estimation
        effort_error_pct = (sifp_estimation_factor - 1) * 100
        
        # Get total SiFP estimated by this model
        if not llm_analysis_df.empty:
            model_row = llm_analysis_df[llm_analysis_df['model'] == model]
            if not model_row.empty:
                model_total_sifp = model_row['total_sifp'].values[0]
                actual_sifp = model_total_sifp / sifp_estimation_factor
                
                # Calculate total effort impact
                estimated_total_effort = model_total_sifp * baseline_hours_per_sifp
                actual_total_effort = actual_sifp * baseline_hours_per_sifp
                total_effort_error = estimated_total_effort - actual_total_effort
                
                # Calculate cost impact using standard rate
                total_cost_impact = total_effort_error * CONFIG['COST_PER_HOUR']
            else:
                model_total_sifp = actual_sifp = total_effort_error = total_cost_impact = 0
        else:
            model_total_sifp = actual_sifp = total_effort_error = total_cost_impact = 0
        
        effort_impact.append({
            'Model': model,
            'LOC_per_SiFP': model_loc_per_sifp,
            'SiFP_Estimation_Factor': sifp_estimation_factor,
            'Desharnais_Hours_per_SiFP': baseline_hours_per_sifp,
            'Effective_Hours_per_Est_SiFP': effective_hours_per_estimated_sifp,
            'Effort_Error_Pct': effort_error_pct,
            'Model_Total_SiFP': model_total_sifp,
            'Actual_SiFP': actual_sifp,
            'Total_Effort_Error_Hours': total_effort_error,
            'Total_Cost_Impact_USD': total_cost_impact
        })
        
        # Display model-specific analysis
        print(f"\n{model}:")
        print(f"  LOC per SiFP: {model_loc_per_sifp:.1f} (baseline: {baseline_loc_per_sifp:.1f})")
        print(f"  SiFP estimation factor: {sifp_estimation_factor:.2f}x")
        print(f"  Interpretation: Model {'overestimates' if sifp_estimation_factor > 1 else 'underestimates'} SiFP count")
        print(f"  Desharnais baseline: {baseline_hours_per_sifp:.2f} hours per actual SiFP")
        print(f"  Effective hours per estimated SiFP: {effective_hours_per_estimated_sifp:.2f}")
        print(f"  Effort estimation error: {effort_error_pct:+.1f}%")
        if model_total_sifp > 0:
            print(f"  Total SiFP estimated: {model_total_sifp:.0f}")
            print(f"  Actual SiFP (implied): {actual_sifp:.0f}")
            print(f"  Total effort error: {total_effort_error:+.0f} hours (${total_cost_impact:+,.0f})")
    
    effort_impact_df = pd.DataFrame(effort_impact)
    
    if not effort_impact_df.empty:
        # Summary statistics
        print("\n\nEffort Impact Summary:")
        print("=" * 40)
        print(f"  Desharnais hours per SiFP: {effort_metrics['avg_hours_per_sifp']:.2f}")
        print(f"  Average SiFP estimation factor: {effort_impact_df['SiFP_Estimation_Factor'].mean():.2f}x")
        print(f"  Average effort error: {effort_impact_df['Effort_Error_Pct'].mean():+.1f}%")
        print(f"  Total cost impact range: ${effort_impact_df['Total_Cost_Impact_USD'].min():,.0f} to ${effort_impact_df['Total_Cost_Impact_USD'].max():,.0f}")
        
        if effort_impact_df['Effort_Error_Pct'].abs().size > 0:
            best_model = effort_impact_df.loc[effort_impact_df['Effort_Error_Pct'].abs().idxmin(), 'Model']
            print(f"  Most accurate effort model: {best_model}")
    
    return effort_impact_df

# Execute effort impact analysis
effort_impact_df = analyze_effort_impact()

print(f"\n✓ Effort impact analysis completed successfully")

In [ ]:
# Cell [8] - Comprehensive Visualization of Normalized Results and Distributions
# Purpose: Create comprehensive visualizations of model performance, accuracy distributions, and cost impacts
# Dependencies: performance_df from Cell 5, effort_impact_df from Cell 7, matplotlib/seaborn from Cell 0
# Breadcrumbs: Setup -> Performance Analysis -> Effort Impact -> Comprehensive Visualization

if 'performance_df' in globals() and 'effort_impact_df' in globals() and not performance_df.empty and not effort_impact_df.empty:
    
    # Get baseline values from previous calculations
    avg_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', industry_metrics.get('LOC_PER_SIFP', 100))
    desharnais_hours_per_sifp = effort_metrics.get('avg_hours_per_sifp', 10)
    project_name = CONFIG.get('NEO4J_PROJECT_NAME', 'Unknown Project')
    
    # Create a larger figure with more subplots
    fig = plt.figure(figsize=(20, 16))
    
    # Define grid for subplots
    gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)
    
    models = performance_df['Model'].values
    x = np.arange(len(models))
    
    # 1. LOC per SiFP Point Comparison
    ax1 = fig.add_subplot(gs[0, :2])
    bars = ax1.bar(x, performance_df['LOC_per_SiFP'], alpha=0.7, color='skyblue')
    ax1.axhline(y=avg_loc_per_sifp, color='red', linestyle='--', 
                label=f'Baseline ({avg_loc_per_sifp:.1f} LOC/SiFP)')
    
    # Color bars based on performance
    for i, bar in enumerate(bars):
        if performance_df.iloc[i]['LOC_per_SiFP'] < avg_loc_per_sifp * 0.8:
            bar.set_color('green')
        elif performance_df.iloc[i]['LOC_per_SiFP'] > avg_loc_per_sifp * 1.2:
            bar.set_color('red')
    
    ax1.set_xlabel('Model')
    ax1.set_ylabel('Lines of Code per SiFP Point')
    ax1.set_title('Code Density per SiFP Point by Model')
    ax1.set_xticks(x)
    ax1.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add success rate and error annotations
    for i, (model, success_rate, error) in enumerate(zip(models, performance_df['Success_Rate'], performance_df['Error_Pct'])):
        ax1.text(i, performance_df.iloc[i]['LOC_per_SiFP'] + 1, 
                f'{success_rate:.0%}\n±{error:.0f}%', ha='center', va='bottom', fontsize=8)
    
    # 2. Model Performance Comparison Table
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.axis('tight')
    ax2.axis('off')
    
    # Create performance summary table
    table_data = []
    for _, row in performance_df.iterrows():
        model_name = row['Model'].split('/')[-1][:20]
        table_data.append([
            model_name,
            f"{row['Success_Rate']:.0%}",
            f"{row['LOC_per_SiFP']:.1f}",
            f"{row['Error_Pct']:.0f}%"
        ])
    
    table = ax2.table(cellText=table_data,
                     colLabels=['Model', 'Success', 'LOC/SiFP', 'Error'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.5)
    ax2.set_title('Model Performance Summary', pad=20)
    
    # 3. Effort per SiFP Point
    ax3 = fig.add_subplot(gs[1, 0])
    bars = ax3.bar(x, effort_impact_df['Effective_Hours_per_Est_SiFP'], 
            color=['green' if x < desharnais_hours_per_sifp else 'orange' 
                   for x in effort_impact_df['Effective_Hours_per_Est_SiFP']], alpha=0.7)
    ax3.axhline(y=desharnais_hours_per_sifp, color='red', linestyle='--', 
                label=f'Desharnais Baseline ({desharnais_hours_per_sifp:.1f} hrs/SiFP)')
    ax3.set_xlabel('Model')
    ax3.set_ylabel('Effective Hours per Estimated SiFP')
    ax3.set_title('Effort Estimation per SiFP Point')
    ax3.set_xticks(x)
    ax3.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. SiFP per KLOC
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.bar(x, performance_df['SiFP_per_KLOC'], alpha=0.7, color='coral')
    ax4.set_xlabel('Model')
    ax4.set_ylabel('SiFP per KLOC')
    ax4.set_title('Function Point Density (SiFP per 1000 LOC)')
    ax4.set_xticks(x)
    ax4.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax4.grid(True, alpha=0.3)
    
    # 5. Total Cost Impact
    ax5 = fig.add_subplot(gs[1, 2])
    bars = ax5.bar(x, effort_impact_df['Total_Cost_Impact_USD'], 
                   color=['darkgreen' if x < 0 else 'darkred' 
                          for x in effort_impact_df['Total_Cost_Impact_USD']], alpha=0.7)
    ax5.set_xlabel('Model')
    ax5.set_ylabel('Total Cost Impact ($)')
    ax5.set_title('Total Cost Impact')
    ax5.set_xticks(x)
    ax5.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax5.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax5.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(effort_impact_df['Total_Cost_Impact_USD']):
        ax5.text(i, v + (1000 if v > 0 else -1000), f'${v:,.0f}', 
                ha='center', va='bottom' if v > 0 else 'top', fontsize=8)
    
    # 6-10. Histograms for each model showing requirement-level accuracy
    histogram_count = 0
    max_histograms = 6  # Limit to 6 histograms to fit in remaining subplot space
    
    for idx, model in enumerate(models[:max_histograms]):
        row_idx = 2 + histogram_count // 3
        col_idx = histogram_count % 3
        
        if row_idx >= 4:  # Don't exceed our grid
            break
            
        ax = fig.add_subplot(gs[row_idx, col_idx])
        
        model_data = llm_estimates_df[llm_estimates_df['model'] == model]
        
        if not model_data.empty:
            model_coverage = model_data['requirement_id'].nunique() / len(ground_truth_requirements)
            
            # Calculate LOC per SiFP for each requirement
            req_loc_per_sifp = []
            for _, req in model_data.iterrows():
                if req['final_sifp'] > 0:
                    est_loc_per_req = (code_summary['total_lines'] * model_coverage) / model_data['requirement_id'].nunique()
                    loc_per_sifp = est_loc_per_req / req['final_sifp']
                    req_loc_per_sifp.append(loc_per_sifp)
            
            if req_loc_per_sifp:
                # Create histogram
                n, bins, patches = ax.hist(req_loc_per_sifp, bins=min(15, len(req_loc_per_sifp)), 
                                         alpha=0.7, color='steelblue', edgecolor='black')
                
                # Color code bins
                for i, patch in enumerate(patches):
                    if i < len(bins) - 1:  # bins has one more element than patches
                        if bins[i] < avg_loc_per_sifp * 0.8:
                            patch.set_facecolor('green')
                        elif bins[i] > avg_loc_per_sifp * 1.2:
                            patch.set_facecolor('red')
                
                # Add baseline line
                ax.axvline(x=avg_loc_per_sifp, color='red', linestyle='--', linewidth=2, 
                          label=f'Baseline: {avg_loc_per_sifp:.1f}')
                ax.axvline(x=np.mean(req_loc_per_sifp), color='blue', linestyle='-', linewidth=2,
                          label=f'Model mean: {np.mean(req_loc_per_sifp):.1f}')
                
                ax.set_xlabel('LOC per SiFP')
                ax.set_ylabel('# Requirements')
                ax.set_title(f'{model.split("/")[-1][:20]}\nAccuracy Distribution')
                ax.legend(fontsize=8)
                ax.grid(True, alpha=0.3)
                
                # Add statistics text
                ax.text(0.95, 0.95, f'n={len(req_loc_per_sifp)}\nσ={np.std(req_loc_per_sifp):.1f}',
                       transform=ax.transAxes, ha='right', va='top', fontsize=8,
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            else:
                ax.text(0.5, 0.5, 'No valid data', transform=ax.transAxes, ha='center', va='center')
                ax.set_title(f'{model.split("/")[-1][:20]}\nNo Data')
        else:
            ax.text(0.5, 0.5, 'No model data', transform=ax.transAxes, ha='center', va='center')
            ax.set_title(f'{model.split("/")[-1][:20]}\nNo Data')
        
        histogram_count += 1
    
    plt.suptitle(f'Comprehensive SiFP Analysis - {project_name}\n'
                 f'All Models Performance and Accuracy Distribution', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    print("✓ Comprehensive visualization completed successfully")
    
else:
    print("Cannot create comprehensive visualization - required data not available")
    print("Available variables:")
    if 'performance_df' in globals():
        print(f"  - performance_df: {len(performance_df) if not performance_df.empty else 'empty'}")
    else:
        print("  - performance_df: not defined")
    
    if 'effort_impact_df' in globals():
        print(f"  - effort_impact_df: {len(effort_impact_df) if not effort_impact_df.empty else 'empty'}")
    else:
        print("  - effort_impact_df: not defined")
    
    if 'baseline_metrics' in globals():
        print("  - baseline_metrics: available")
    else:
        print("  - baseline_metrics: not defined")

In [ ]:
# Cell [10] - Executive Summary with Complete UFP→SiFP→LOC→Effort Analysis  
# Purpose: Generate comprehensive executive summary with complete conversion chain analysis and business insights
# Dependencies: All previous analysis results, CONFIG settings, comprehensive metrics from entire workflow
# Breadcrumbs: Setup -> Analysis -> Recommendations -> Executive Summary & Business Impact Report

print("EXECUTIVE SUMMARY - NORMALIZED SIFP ANALYSIS")
print("=" * 60)

# Get project name safely
project_name = CONFIG.get('NEO4J_PROJECT_NAME', 'Unknown Project') if 'CONFIG' in globals() else 'Unknown Project'
print(f"Project: {project_name}")
print(f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")

if 'performance_df' in globals() and 'effort_impact_df' in globals() and not performance_df.empty and not effort_impact_df.empty:
    print(f"\nData Summary:")
    
    if 'code_metrics_df' in globals():
        print(f"  Total code files in project: {len(code_metrics_df)}")
        print(f"  Total lines of code in project: {code_metrics_df['CountLineCode'].sum():,}")
    else:
        print("  Code metrics: Not available")
    
    if 'ground_truth_requirements' in globals():
        print(f"  Ground truth requirements: {len(ground_truth_requirements)}")
    else:
        print("  Ground truth requirements: Not available")
    
    if 'llm_estimates_df' in globals() and not llm_estimates_df.empty:
        estimated_requirements = llm_estimates_df['requirement_id'].unique()
        if 'ground_truth_requirements' in globals():
            print(f"  Requirements with estimates: {len(estimated_requirements)} ({len(estimated_requirements)/len(ground_truth_requirements):.1%})")
        else:
            print(f"  Requirements with estimates: {len(estimated_requirements)}")
    
    # Get baseline values safely
    conversion_factor = CONFIG.get('CONVERSION_FACTOR', 0.957) if 'CONFIG' in globals() else 0.957
    
    # Get industry and project baselines
    industry_loc_per_sifp = industry_metrics.get('LOC_PER_SIFP', 100) if 'industry_metrics' in globals() else 100
    project_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', industry_loc_per_sifp) if 'baseline_metrics' in globals() else industry_loc_per_sifp
    desharnais_hours_per_sifp = effort_metrics.get('avg_hours_per_sifp', 10) if 'effort_metrics' in globals() else 10
    
    print(f"\nConversion Factors and Baselines:")
    print(f"  UFP → SiFP: {conversion_factor} (from Desharnais research)")
    print(f"  SiFP → Effort: {desharnais_hours_per_sifp:.2f} hours/SiFP (Desharnais dataset)")
    print(f"  SiFP → LOC: {project_loc_per_sifp:.1f} LOC/SiFP (project weighted average)")
    print(f"  Industry baseline: {industry_loc_per_sifp:.1f} LOC/SiFP")
    print(f"  Project vs Industry: {(project_loc_per_sifp - industry_loc_per_sifp)/industry_loc_per_sifp*100:+.1f}%")
    
    # Detailed performance for each model
    print(f"\n" + "="*80)
    print("DETAILED MODEL PERFORMANCE")
    print("="*80)
    
    for idx, row in performance_df.iterrows():
        model = row['Model']
        effort_row = effort_impact_df[effort_impact_df['Model'] == model]
        
        if not effort_row.empty:
            effort_row = effort_row.iloc[0]
            
            if 'llm_analysis_df' in globals() and not llm_analysis_df.empty:
                llm_row = llm_analysis_df[llm_analysis_df['model'] == model]
                if not llm_row.empty:
                    llm_row = llm_row.iloc[0]
                else:
                    llm_row = None
            else:
                llm_row = None
            
            print(f"\n{idx+1}. {model}")
            print("-" * len(f"{idx+1}. {model}"))
            
            print(f"\n  Estimation Coverage:")
            print(f"    - Success rate: {row['Success_Rate']:.1%}")
            if llm_row is not None:
                print(f"    - Requirements estimated: {llm_row['successful_reqs']}/{len(ground_truth_requirements) if 'ground_truth_requirements' in globals() else 'unknown'}")
            
            print(f"\n  SiFP Estimates:")
            if llm_row is not None:
                print(f"    - Total SiFP: {llm_row['total_sifp']:.0f}")
                print(f"    - Equivalent UFP: {llm_row['equivalent_ufp']:.0f}")
                print(f"    - Average SiFP per requirement: {row['Avg_SiFP_per_Req']:.1f}")
            else:
                print(f"    - Average SiFP per requirement: {row.get('Avg_SiFP_per_Req', 'N/A')}")
            
            print(f"\n  Accuracy Metrics:")
            print(f"    - LOC per SiFP: {row['LOC_per_SiFP']:.1f} (baseline: {project_loc_per_sifp:.1f})")
            print(f"    - Error: {row.get('LOC_per_SiFP_Error', 'N/A'):+.1f} LOC/SiFP ({row['Error_Pct']:+.1f}%)")
            print(f"    - SiFP estimation factor: {effort_row['SiFP_Estimation_Factor']:.2f}x")
            
            print(f"\n  Effort Impact:")
            print(f"    - Desharnais baseline: {desharnais_hours_per_sifp:.2f} hours/SiFP")
            print(f"    - Effort estimation error: {effort_row['Effort_Error_Pct']:+.1f}%")
            print(f"    - Total effort error: {effort_row['Total_Effort_Error_Hours']:+.0f} hours")
            print(f"    - Cost impact: ${effort_row['Total_Cost_Impact_USD']:+,.0f}")
            
            print(f"\n  Quality Indicators:")
            print(f"    - Average confidence: {row.get('Avg_Confidence', 0):.1%}")
            print(f"    - Average judge score: {row.get('Avg_Judge_Score', 0):.2f}/5")
    
    # Analysis of the conversion chain
    print(f"\n" + "="*80)
    print("CONVERSION CHAIN ANALYSIS")
    print("="*80)
    
    print(f"\nFor a typical requirement in this project:")
    if 'llm_analysis_df' in globals() and not llm_analysis_df.empty:
        avg_sifp_per_req = llm_analysis_df['sifp_per_req'].mean()
        print(f"  Average SiFP per requirement: {avg_sifp_per_req:.1f}")
        print(f"  Equivalent UFP: {avg_sifp_per_req / conversion_factor:.1f}")
        print(f"  Expected LOC: {avg_sifp_per_req * project_loc_per_sifp:.0f}")
        print(f"  Expected effort: {avg_sifp_per_req * desharnais_hours_per_sifp:.0f} hours")
    else:
        print("  Analysis not available - LLM analysis data missing")
    
    # Recommendations
    print(f"\n" + "="*80)
    print("RECOMMENDATIONS")
    print("="*80)
    
    if not performance_df.empty:
        best_accuracy = performance_df.loc[performance_df['Error_Pct'].idxmin()]['Model']
        print(f"\n1. For most accurate code size estimation: {best_accuracy}")
    else:
        print("\n1. For most accurate code size estimation: Data not available")
    
    if not effort_impact_df.empty:
        best_effort = effort_impact_df.loc[effort_impact_df['Effort_Error_Pct'].abs().idxmin()]['Model']
        print(f"2. For most accurate effort estimation: {best_effort}")
    else:
        print("2. For most accurate effort estimation: Data not available")
    
    print(f"3. Use Desharnais baseline of {desharnais_hours_per_sifp:.1f} hours per SiFP for effort planning")
    print(f"4. Apply UFP conversion factor of {conversion_factor} when comparing to UFP-based estimates")
    print(f"5. Consider that this project has {(project_loc_per_sifp - industry_loc_per_sifp)/industry_loc_per_sifp*100:+.1f}% different LOC/SiFP than industry average")
    
    # Save results with all conversion factors
    try:
        os.makedirs('results', exist_ok=True)
        
        # Create comprehensive summary
        conversion_summary = pd.DataFrame({
            'Metric': ['UFP→SiFP Factor', 'Industry LOC/SiFP', 'Project LOC/SiFP', 'Desharnais Hours/SiFP'],
            'Value': [conversion_factor, industry_loc_per_sifp, project_loc_per_sifp, desharnais_hours_per_sifp]
        })
        conversion_summary.to_csv(f'results/conversion_factors_{project_name}.csv', index=False)
        
        # Save all other results
        performance_df.to_csv(f'results/normalized_performance_{project_name}.csv', index=False)
        effort_impact_df.to_csv(f'results/normalized_effort_impact_{project_name}.csv', index=False)
        
        if 'llm_analysis_df' in globals() and not llm_analysis_df.empty:
            llm_analysis_df.to_csv(f'results/normalized_llm_analysis_{project_name}.csv', index=False)
        
        print(f"\n✓ Results saved to results/ directory")
        
    except Exception as e:
        print(f"\nWarning: Could not save results - {e}")

else:
    print("Missing required data for executive summary")
    print("Available data:")
    
    if 'performance_df' in globals():
        print(f"  - performance_df: {len(performance_df) if not performance_df.empty else 'empty'}")
    else:
        print("  - performance_df: not available")
    
    if 'effort_impact_df' in globals():
        print(f"  - effort_impact_df: {len(effort_impact_df) if not effort_impact_df.empty else 'empty'}")
    else:
        print("  - effort_impact_df: not available")
    
    if 'llm_analysis_df' in globals():
        print(f"  - llm_analysis_df: {len(llm_analysis_df) if not llm_analysis_df.empty else 'empty'}")
    else:
        print("  - llm_analysis_df: not available")

print(f"\n✓ Executive summary completed")

In [ ]:
# Cell [11] - Statistical Hypothesis Testing for >30% Improvement in Time-to-Market
# Purpose: Perform formal statistical testing to validate hypothesis about hallucination-reducing techniques
# Dependencies: performance_df, effort_impact_df, scipy.stats for statistical tests
# Breadcrumbs: Setup -> Analysis -> Executive Summary -> Statistical Hypothesis Testing

from scipy.stats import ttest_ind, mannwhitneyu, chi2_contingency
from scipy import stats
import numpy as np

def perform_hypothesis_testing():
    """
    Perform formal statistical hypothesis testing for the >30% improvement claim
    
    Returns:
        dict: Statistical test results including p-values and confidence intervals
    """
    
    print("STATISTICAL HYPOTHESIS TESTING")
    print("=" * 60)
    print("HYPOTHESIS: Implementing hallucination-reducing techniques in LLMs")
    print("significantly improve (>30%) time to market in new product development")
    print("\nOPERATIONAL DEFINITIONS:")
    print("- Hallucination-reducing techniques: Multi-stage refinement (actor→judge→meta-judge)")
    print("- Time-to-market improvement: Measured via estimation accuracy reducing project delays")
    print("- Significance threshold: >30% improvement with p < 0.05")
    
    if performance_df.empty or effort_impact_df.empty:
        print("\nWarning: Insufficient data for statistical testing")
        return {}
    
    # Define treatment vs control groups based on model characteristics
    # Assumption: Models with judge scores > 3.5 represent "hallucination-reducing" techniques
    treatment_threshold = 3.5
    
    # Categorize models
    treatment_models = performance_df[performance_df['Avg_Judge_Score'] > treatment_threshold]
    control_models = performance_df[performance_df['Avg_Judge_Score'] <= treatment_threshold]
    
    print(f"\nGROUP DEFINITIONS:")
    print(f"Treatment group (Judge Score > {treatment_threshold}): {len(treatment_models)} models")
    print(f"Control group (Judge Score ≤ {treatment_threshold}): {len(control_models)} models")
    
    if len(treatment_models) == 0 or len(control_models) == 0:
        print("\nWarning: Insufficient models in treatment or control groups for comparison")
        print("Adjusting criteria...")
        
        # Alternative grouping: Top 50% vs bottom 50% by judge score
        median_judge_score = performance_df['Avg_Judge_Score'].median()
        treatment_models = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
        control_models = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
        
        print(f"Alternative grouping by median judge score ({median_judge_score:.2f}):")
        print(f"Treatment group: {len(treatment_models)} models")
        print(f"Control group: {len(control_models)} models")
    
    # Primary outcome: Estimation accuracy (lower error = better time-to-market)
    treatment_errors = treatment_models['Error_Pct'].values
    control_errors = control_models['Error_Pct'].values
    
    # Secondary outcomes: Success rate, effort estimation accuracy
    treatment_success = treatment_models['Success_Rate'].values
    control_success = control_models['Success_Rate'].values
    
    print(f"\nDESCRIPTIVE STATISTICS:")
    print(f"Treatment Group (n={len(treatment_errors)}):")
    print(f"  Mean error: {np.mean(treatment_errors):.2f}% (±{np.std(treatment_errors):.2f})")
    print(f"  Mean success rate: {np.mean(treatment_success):.2%} (±{np.std(treatment_success):.2%})")
    
    print(f"\nControl Group (n={len(control_errors)}):")
    print(f"  Mean error: {np.mean(control_errors):.2f}% (±{np.std(control_errors):.2f})")
    print(f"  Mean success rate: {np.mean(control_success):.2%} (±{np.std(control_success):.2%})")
    
    # Calculate improvement percentages
    error_improvement = (np.mean(control_errors) - np.mean(treatment_errors)) / np.mean(control_errors) * 100
    success_improvement = (np.mean(treatment_success) - np.mean(control_success)) / np.mean(control_success) * 100
    
    print(f"\nIMPROVEMENT ANALYSIS:")
    print(f"  Error reduction: {error_improvement:+.1f}%")
    print(f"  Success rate improvement: {success_improvement:+.1f}%")
    print(f"  Meets >30% threshold: {'YES' if abs(error_improvement) > 30 or success_improvement > 30 else 'NO'}")
    
    # Statistical tests
    test_results = {}
    
    # 1. T-test for estimation errors (assuming normal distribution)
    if len(treatment_errors) > 1 and len(control_errors) > 1:
        t_stat, t_pvalue = ttest_ind(treatment_errors, control_errors)
        
        # Calculate confidence interval for difference
        pooled_se = np.sqrt(np.var(treatment_errors)/len(treatment_errors) + 
                           np.var(control_errors)/len(control_errors))
        mean_diff = np.mean(treatment_errors) - np.mean(control_errors)
        margin_error = 1.96 * pooled_se  # 95% CI
        ci_lower = mean_diff - margin_error
        ci_upper = mean_diff + margin_error
        
        test_results['t_test'] = {
            'statistic': t_stat,
            'p_value': t_pvalue,
            'mean_difference': mean_diff,
            'ci_95': (ci_lower, ci_upper),
            'significant': t_pvalue < 0.05
        }
    
    # 2. Mann-Whitney U test (non-parametric alternative)
    if len(treatment_errors) > 1 and len(control_errors) > 1:
        u_stat, u_pvalue = mannwhitneyu(treatment_errors, control_errors, alternative='two-sided')
        
        test_results['mann_whitney'] = {
            'statistic': u_stat,
            'p_value': u_pvalue,
            'significant': u_pvalue < 0.05
        }
    
    # 3. Effect size (Cohen's d)
    if len(treatment_errors) > 1 and len(control_errors) > 1:
        pooled_std = np.sqrt(((len(treatment_errors)-1)*np.var(treatment_errors) + 
                             (len(control_errors)-1)*np.var(control_errors)) / 
                            (len(treatment_errors) + len(control_errors) - 2))
        cohens_d = (np.mean(treatment_errors) - np.mean(control_errors)) / pooled_std
        
        test_results['effect_size'] = {
            'cohens_d': cohens_d,
            'interpretation': 'small' if abs(cohens_d) < 0.5 else 'medium' if abs(cohens_d) < 0.8 else 'large'
        }
    
    # Display statistical test results
    print(f"\nSTATISTICAL TEST RESULTS:")
    print("=" * 40)
    
    if 't_test' in test_results:
        t_result = test_results['t_test']
        print(f"\n1. Independent Samples T-Test:")
        print(f"   H₀: No difference in estimation errors between groups")
        print(f"   H₁: Significant difference exists")
        print(f"   t-statistic: {t_result['statistic']:.3f}")
        print(f"   p-value: {t_result['p_value']:.4f}")
        print(f"   Mean difference: {t_result['mean_difference']:.2f}% error")
        print(f"   95% CI: ({t_result['ci_95'][0]:.2f}, {t_result['ci_95'][1]:.2f})")
        print(f"   Significant: {'YES' if t_result['significant'] else 'NO'} (α = 0.05)")
    
    if 'mann_whitney' in test_results:
        u_result = test_results['mann_whitney']
        print(f"\n2. Mann-Whitney U Test (Non-parametric):")
        print(f"   U-statistic: {u_result['statistic']:.3f}")
        print(f"   p-value: {u_result['p_value']:.4f}")
        print(f"   Significant: {'YES' if u_result['significant'] else 'NO'} (α = 0.05)")
    
    if 'effect_size' in test_results:
        effect = test_results['effect_size']
        print(f"\n3. Effect Size Analysis:")
        print(f"   Cohen's d: {effect['cohens_d']:.3f}")
        print(f"   Interpretation: {effect['interpretation']} effect")
    
    # Power analysis (post-hoc)
    if 'effect_size' in test_results and len(treatment_errors) > 1:
        from scipy.stats import norm
        alpha = 0.05
        n1, n2 = len(treatment_errors), len(control_errors)
        effect_size = abs(test_results['effect_size']['cohens_d'])
        
        # Simplified power calculation
        se = np.sqrt(1/n1 + 1/n2)
        critical_t = norm.ppf(1 - alpha/2)
        power = 1 - norm.cdf(critical_t - effect_size/se) + norm.cdf(-critical_t - effect_size/se)
        
        print(f"\n4. Statistical Power Analysis:")
        print(f"   Observed power: {power:.3f}")
        print(f"   Sample size (treatment): {n1}")
        print(f"   Sample size (control): {n2}")
        print(f"   Power interpretation: {'Adequate' if power > 0.8 else 'Inadequate'} (target: 0.8)")
    
    # Conclusion for hypothesis
    print(f"\nHYPOTHESIS TESTING CONCLUSION:")
    print("=" * 50)
    
    significant_improvement = (abs(error_improvement) > 30 or success_improvement > 30)
    statistically_significant = (test_results.get('t_test', {}).get('significant', False) or 
                               test_results.get('mann_whitney', {}).get('significant', False))
    
    print(f"\n✓ Magnitude Test: {'PASS' if significant_improvement else 'FAIL'}")
    print(f"  - Required: >30% improvement")
    print(f"  - Observed: {max(abs(error_improvement), success_improvement):.1f}% improvement")
    
    print(f"\n✓ Statistical Significance: {'PASS' if statistically_significant else 'FAIL'}")
    print(f"  - Required: p < 0.05")
    print(f"  - Observed: p = {test_results.get('t_test', {}).get('p_value', 'N/A')}")
    
    hypothesis_supported = significant_improvement and statistically_significant
    
    print(f"\n🎯 FINAL VERDICT: {'HYPOTHESIS SUPPORTED' if hypothesis_supported else 'HYPOTHESIS NOT SUPPORTED'}")
    
    if not hypothesis_supported:
        print(f"\nRECOMMENDATIONS FOR FUTURE RESEARCH:")
        print(f"  1. Increase sample size (current: {len(performance_df)} models)")
        print(f"  2. Define clearer treatment/control groups")
        print(f"  3. Collect direct time-to-market measurements")
        print(f"  4. Implement randomized controlled trial design")
        print(f"  5. Establish baseline measurements before intervention")
    
    return test_results

# Execute statistical hypothesis testing
if 'performance_df' in globals() and not performance_df.empty:
    statistical_results = perform_hypothesis_testing()
else:
    print("Cannot perform hypothesis testing - performance data not available")
    statistical_results = {}


In [ ]:
# Cell [12] - Statistical Validation Visualizations and Bootstrap Analysis
# Purpose: Create visualizations for statistical tests and perform bootstrap confidence intervals
# Dependencies: statistical_results from Cell 11, matplotlib, seaborn, bootstrap methods
# Breadcrumbs: Setup -> Analysis -> Statistical Testing -> Validation Visualizations

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap
import numpy as np

def create_statistical_visualizations():
    """
    Create visualizations to support statistical hypothesis testing
    """
    
    if performance_df.empty:
        print("No data available for statistical visualizations")
        return
    
    # Set up the figure
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Statistical Validation of Hallucination-Reducing Techniques\nHypothesis: >30% Improvement in Time-to-Market', fontsize=16)
    
    # Define groups based on judge scores (using median split)
    median_judge_score = performance_df['Avg_Judge_Score'].median()
    treatment_group = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
    control_group = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
    
    # 1. Box plot comparison of error rates
    ax1 = axes[0, 0]
    data_for_box = [control_group['Error_Pct'].values, treatment_group['Error_Pct'].values]
    box_plot = ax1.boxplot(data_for_box, labels=['Control\n(Low Judge Score)', 'Treatment\n(High Judge Score)'], patch_artist=True)
    box_plot['boxes'][0].set_facecolor('lightcoral')
    box_plot['boxes'][1].set_facecolor('lightgreen')
    ax1.set_ylabel('Estimation Error (%)')
    ax1.set_title('Error Rate Distribution by Group')
    ax1.grid(True, alpha=0.3)
    
    # Add significance indicator if available
    if 'statistical_results' in globals() and 't_test' in statistical_results:
        p_val = statistical_results['t_test']['p_value']
        ax1.text(0.5, ax1.get_ylim()[1] * 0.9, f'p = {p_val:.4f}', 
                ha='center', transform=ax1.transData, fontsize=12,
                bbox=dict(boxstyle='round', facecolor='yellow' if p_val < 0.05 else 'white'))
    
    # 2. Success rate comparison
    ax2 = axes[0, 1]
    success_data = [control_group['Success_Rate'].values, treatment_group['Success_Rate'].values]
    box_plot2 = ax2.boxplot(success_data, labels=['Control', 'Treatment'], patch_artist=True)
    box_plot2['boxes'][0].set_facecolor('lightcoral')
    box_plot2['boxes'][1].set_facecolor('lightgreen')
    ax2.set_ylabel('Success Rate')
    ax2.set_title('Success Rate Distribution by Group')
    ax2.grid(True, alpha=0.3)
    
    # 3. Confidence intervals for improvement
    ax3 = axes[0, 2]
    
    # Bootstrap confidence intervals
    def bootstrap_improvement(control_data, treatment_data, n_bootstrap=1000):
        improvements = []
        for _ in range(n_bootstrap):
            # Resample with replacement
            control_sample = np.random.choice(control_data, len(control_data), replace=True)
            treatment_sample = np.random.choice(treatment_data, len(treatment_data), replace=True)
            
            # Calculate improvement
            improvement = (np.mean(control_sample) - np.mean(treatment_sample)) / np.mean(control_sample) * 100
            improvements.append(improvement)
        
        return np.array(improvements)
    
    if len(control_group) > 0 and len(treatment_group) > 0:
        # Bootstrap for error improvement
        error_improvements = bootstrap_improvement(control_group['Error_Pct'].values, 
                                                 treatment_group['Error_Pct'].values)
        
        # Bootstrap for success rate improvement  
        control_success_neg = np.array(control_group['Success_Rate'].values) * (-1)
        treatment_success_neg = np.array(treatment_group['Success_Rate'].values) * (-1)
        success_improvements = bootstrap_improvement(control_success_neg, treatment_success_neg) * (-1)
        
        # Plot histograms
        ax3.hist(error_improvements, bins=20, alpha=0.7, label='Error Reduction', color='skyblue', density=True)
        ax3.hist(success_improvements, bins=20, alpha=0.7, label='Success Improvement', color='lightgreen', density=True)
        
        # Add 30% threshold line
        ax3.axvline(x=30, color='red', linestyle='--', linewidth=2, label='30% Threshold')
        ax3.axvline(x=-30, color='red', linestyle='--', linewidth=2)
        
        # Calculate and display confidence intervals
        error_ci = np.percentile(error_improvements, [2.5, 97.5])
        success_ci = np.percentile(success_improvements, [2.5, 97.5])
        
        ax3.text(0.05, 0.95, f'Error Reduction CI: ({error_ci[0]:.1f}%, {error_ci[1]:.1f}%)', 
                transform=ax3.transAxes, verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        ax3.text(0.05, 0.85, f'Success Improve CI: ({success_ci[0]:.1f}%, {success_ci[1]:.1f}%)', 
                transform=ax3.transAxes, verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax3.set_xlabel('Improvement (%)')
        ax3.set_ylabel('Density')
        ax3.set_title('Bootstrap Confidence Intervals\nfor Improvement Metrics')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    # 4. Scatter plot of judge score vs accuracy
    ax4 = axes[1, 0]
    scatter = ax4.scatter(performance_df['Avg_Judge_Score'], performance_df['Error_Pct'], 
                         c=performance_df['Success_Rate'], cmap='RdYlGn', s=100, alpha=0.7)
    
    # Add trend line
    z = np.polyfit(performance_df['Avg_Judge_Score'], performance_df['Error_Pct'], 1)
    p = np.poly1d(z)
    ax4.plot(performance_df['Avg_Judge_Score'], p(performance_df['Avg_Judge_Score']), 
             "r--", alpha=0.8, linewidth=2)
    
    # Add correlation coefficient
    corr_coef = performance_df['Avg_Judge_Score'].corr(performance_df['Error_Pct'])
    ax4.text(0.05, 0.95, f'Correlation: r = {corr_coef:.3f}', 
            transform=ax4.transAxes, verticalalignment='top', fontsize=12,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax4.set_xlabel('Average Judge Score')
    ax4.set_ylabel('Estimation Error (%)')
    ax4.set_title('Judge Score vs Estimation Accuracy')
    plt.colorbar(scatter, ax=ax4, label='Success Rate')
    ax4.grid(True, alpha=0.3)
    
    # 5. Power analysis visualization
    ax5 = axes[1, 1]
    
    # Simulate power curves for different sample sizes
    effect_sizes = np.linspace(0, 2, 50)
    sample_sizes = [5, 10, 20, 50]
    
    for n in sample_sizes:
        # Simplified power calculation
        powers = []
        for effect in effect_sizes:
            # Using non-central t-distribution approximation
            from scipy.stats import norm
            critical_t = norm.ppf(0.975)  # Two-tailed test, alpha = 0.05
            se = np.sqrt(2/n)  # Standard error for equal group sizes
            power = 1 - norm.cdf(critical_t - effect/se) + norm.cdf(-critical_t - effect/se)
            powers.append(power)
        
        ax5.plot(effect_sizes, powers, label=f'n = {n} per group', linewidth=2)
    
    ax5.axhline(y=0.8, color='red', linestyle='--', label='Target Power (0.8)')
    ax5.axvline(x=0.8, color='orange', linestyle='--', label='Medium Effect Size')
    ax5.set_xlabel('Effect Size (Cohen\'s d)')
    ax5.set_ylabel('Statistical Power')
    ax5.set_title('Power Analysis: Sample Size Requirements')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    ax5.set_xlim(0, 2)
    ax5.set_ylim(0, 1)
    
    # 6. Practical significance vs statistical significance
    ax6 = axes[1, 2]
    
    # Create a significance matrix
    models = performance_df['Model'].str.split('/').str[-1].str[:15]  # Shorten model names
    y_pos = np.arange(len(models))
    
    # Calculate improvement for each model relative to mean
    mean_error = performance_df['Error_Pct'].mean()
    improvements = (mean_error - performance_df['Error_Pct']) / mean_error * 100
    
    # Color code based on practical and statistical significance
    colors = []
    for imp in improvements:
        if abs(imp) > 30:  # Practically significant
            colors.append('green')
        elif abs(imp) > 15:  # Moderate improvement
            colors.append('orange') 
        else:  # Small improvement
            colors.append('red')
    
    bars = ax6.barh(y_pos, improvements, color=colors, alpha=0.7)
    ax6.set_yticks(y_pos)
    ax6.set_yticklabels(models, fontsize=8)
    ax6.set_xlabel('Improvement vs Mean (%)')
    ax6.set_title('Practical Significance Analysis\n(Green: >30%, Orange: >15%, Red: <15%)')
    ax6.axvline(x=30, color='green', linestyle='--', linewidth=2, label='>30% Threshold')
    ax6.axvline(x=-30, color='green', linestyle='--', linewidth=2)
    ax6.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics table
    print("\nSTATISTICAL VALIDATION SUMMARY")
    print("=" * 50)
    
    if len(control_group) > 0 and len(treatment_group) > 0:
        print(f"\nGroup Comparisons:")
        print(f"Control Group (n={len(control_group)}):")
        print(f"  Mean Error: {control_group['Error_Pct'].mean():.2f}% ± {control_group['Error_Pct'].std():.2f}")
        print(f"  Mean Success: {control_group['Success_Rate'].mean():.2%} ± {control_group['Success_Rate'].std():.2%}")
        
        print(f"\nTreatment Group (n={len(treatment_group)}):")
        print(f"  Mean Error: {treatment_group['Error_Pct'].mean():.2f}% ± {treatment_group['Error_Pct'].std():.2f}")
        print(f"  Mean Success: {treatment_group['Success_Rate'].mean():.2%} ± {treatment_group['Success_Rate'].std():.2%}")
        
        # Effect sizes
        error_effect_size = (treatment_group['Error_Pct'].mean() - control_group['Error_Pct'].mean()) / performance_df['Error_Pct'].std()
        success_effect_size = (treatment_group['Success_Rate'].mean() - control_group['Success_Rate'].mean()) / performance_df['Success_Rate'].std()
        
        print(f"\nEffect Sizes:")
        print(f"  Error Reduction: d = {-error_effect_size:.3f}")  # Negative because lower error is better
        print(f"  Success Improvement: d = {success_effect_size:.3f}")
        
        # Practical significance assessment
        error_improvement = (control_group['Error_Pct'].mean() - treatment_group['Error_Pct'].mean()) / control_group['Error_Pct'].mean() * 100
        success_improvement = (treatment_group['Success_Rate'].mean() - control_group['Success_Rate'].mean()) / control_group['Success_Rate'].mean() * 100
        
        print(f"\nPractical Significance Assessment:")
        print(f"  Error Reduction: {error_improvement:+.1f}% ({'SIGNIFICANT' if abs(error_improvement) > 30 else 'NOT SIGNIFICANT'})")
        print(f"  Success Improvement: {success_improvement:+.1f}% ({'SIGNIFICANT' if success_improvement > 30 else 'NOT SIGNIFICANT'})")

# Execute statistical visualizations
if 'performance_df' in globals() and not performance_df.empty:
    create_statistical_visualizations()
else:
    print("Cannot create statistical visualizations - performance data not available")

In [ ]:
# Cell [13] - Final Executive Summary with Complete UFP→SiFP→LOC→Effort Analysis  
# Purpose: Generate comprehensive executive summary with complete conversion chain analysis and business insights
# Dependencies: All previous analysis results, CONFIG settings, comprehensive metrics from entire workflow
# Breadcrumbs: Setup -> Analysis -> Recommendations -> Final Executive Summary & Business Impact Report

print("EXECUTIVE SUMMARY - NORMALIZED SIFP ANALYSIS")
print("=" * 60)

# Get project name safely
project_name = CONFIG.get('NEO4J_PROJECT_NAME', 'Unknown Project') if 'CONFIG' in globals() else 'Unknown Project'
print(f"Project: {project_name}")
print(f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")

if 'performance_df' in globals() and 'effort_impact_df' in globals() and not performance_df.empty and not effort_impact_df.empty:
    print(f"\nData Summary:")
    
    if 'code_metrics_df' in globals():
        print(f"  Total code files in project: {len(code_metrics_df)}")
        print(f"  Total lines of code in project: {code_metrics_df['CountLineCode'].sum():,}")
    else:
        print("  Code metrics: Not available")
    
    if 'ground_truth_requirements' in globals():
        print(f"  Ground truth requirements: {len(ground_truth_requirements)}")
    else:
        print("  Ground truth requirements: Not available")
    
    if 'llm_estimates_df' in globals() and not llm_estimates_df.empty:
        estimated_requirements = llm_estimates_df['requirement_id'].unique()
        if 'ground_truth_requirements' in globals():
            print(f"  Requirements with estimates: {len(estimated_requirements)} ({len(estimated_requirements)/len(ground_truth_requirements):.1%})")
        else:
            print(f"  Requirements with estimates: {len(estimated_requirements)}")
    
    # Get baseline values safely
    conversion_factor = CONFIG.get('CONVERSION_FACTOR', 0.957) if 'CONFIG' in globals() else 0.957
    
    # Get industry and project baselines
    industry_loc_per_sifp = industry_metrics.get('LOC_PER_SIFP', 100) if 'industry_metrics' in globals() else 100
    project_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', industry_loc_per_sifp) if 'baseline_metrics' in globals() else industry_loc_per_sifp
    desharnais_hours_per_sifp = effort_metrics.get('avg_hours_per_sifp', 10) if 'effort_metrics' in globals() else 10
    
    print(f"\nConversion Factors and Baselines:")
    print(f"  UFP → SiFP: {conversion_factor} (from Desharnais research)")
    print(f"  SiFP → Effort: {desharnais_hours_per_sifp:.2f} hours/SiFP (Desharnais dataset)")
    print(f"  SiFP → LOC: {project_loc_per_sifp:.1f} LOC/SiFP (project weighted average)")
    print(f"  Industry baseline: {industry_loc_per_sifp:.1f} LOC/SiFP")
    print(f"  Project vs Industry: {(project_loc_per_sifp - industry_loc_per_sifp)/industry_loc_per_sifp*100:+.1f}%")
    
    # Statistical hypothesis testing summary
    print(f"\n" + "="*80)
    print("STATISTICAL HYPOTHESIS TESTING SUMMARY")
    print("="*80)
    
    if 'statistical_results' in globals() and statistical_results:
        print(f"\nHypothesis: Hallucination-reducing techniques improve time-to-market by >30%")
        print(f"Operational Definition: Multi-stage refinement (actor→judge→meta-judge)")
        
        # Check if we have t-test results
        if 't_test' in statistical_results:
            t_result = statistical_results['t_test']
            print(f"\nStatistical Test Results:")
            print(f"  t-statistic: {t_result['statistic']:.3f}")
            print(f"  p-value: {t_result['p_value']:.4f}")
            print(f"  95% Confidence Interval: ({t_result['ci_95'][0]:.2f}, {t_result['ci_95'][1]:.2f})")
            print(f"  Statistically significant: {'YES' if t_result['significant'] else 'NO'} (α = 0.05)")
        
        # Check effect size
        if 'effect_size' in statistical_results:
            effect = statistical_results['effect_size']
            print(f"\nEffect Size Analysis:")
            print(f"  Cohen's d: {effect['cohens_d']:.3f}")
            print(f"  Effect size interpretation: {effect['interpretation']}")
        
        print(f"\nHypothesis Testing Conclusion:")
        print(f"  The statistical analysis provides evidence for evaluating the >30% improvement claim")
        print(f"  Results should be interpreted in context of sample size and study design limitations")
    else:
        print(f"\nStatistical hypothesis testing was not completed successfully")
        print(f"Additional data or larger sample sizes may be needed for formal hypothesis testing")
    
    # Model recommendations
    print(f"\n" + "="*80)
    print("MODEL RECOMMENDATIONS")
    print("="*80)
    
    if not performance_df.empty:
        # Best performing models
        best_accuracy = performance_df.loc[performance_df['Error_Pct'].idxmin()]
        best_coverage = performance_df.loc[performance_df['Success_Rate'].idxmax()]
        
        print(f"\n1. MOST ACCURATE MODEL:")
        print(f"   Model: {best_accuracy['Model']}")
        print(f"   Error rate: {best_accuracy['Error_Pct']:.1f}%")
        print(f"   Success rate: {best_accuracy['Success_Rate']:.1%}")
        print(f"   LOC per SiFP: {best_accuracy['LOC_per_SiFP']:.1f}")
        
        print(f"\n2. BEST COVERAGE MODEL:")
        print(f"   Model: {best_coverage['Model']}")
        print(f"   Success rate: {best_coverage['Success_Rate']:.1%}")
        print(f"   Error rate: {best_coverage['Error_Pct']:.1f}%")
        print(f"   LOC per SiFP: {best_coverage['LOC_per_SiFP']:.1f}")
        
        if not effort_impact_df.empty:
            best_effort = effort_impact_df.loc[effort_impact_df['Effort_Error_Pct'].abs().idxmin()]
            print(f"\n3. BEST EFFORT ESTIMATION MODEL:")
            print(f"   Model: {best_effort['Model']}")
            print(f"   Effort error: {best_effort['Effort_Error_Pct']:+.1f}%")
            print(f"   Cost impact: ${best_effort['Total_Cost_Impact_USD']:+,.0f}")
    
    # Business impact summary
    print(f"\n" + "="*80)
    print("BUSINESS IMPACT SUMMARY")
    print("="*80)
    
    if not effort_impact_df.empty:
        total_cost_impact = effort_impact_df['Total_Cost_Impact_USD'].sum()
        avg_effort_error = effort_impact_df['Effort_Error_Pct'].mean()
        max_cost_impact = effort_impact_df['Total_Cost_Impact_USD'].abs().max()
        
        print(f"\nCost Impact Analysis:")
        print(f"  Total net cost impact: ${total_cost_impact:+,.0f}")
        print(f"  Average effort estimation error: {avg_effort_error:+.1f}%")
        print(f"  Maximum cost impact (single model): ${max_cost_impact:,.0f}")
        print(f"  Cost per hour assumed: ${CONFIG.get('COST_PER_HOUR', 100)}/hour")
        
        print(f"\nKey Business Insights:")
        if abs(avg_effort_error) < 15:
            print(f"  ✓ Average estimation errors are within acceptable range (<15%)")
        else:
            print(f"  ⚠ Average estimation errors exceed 15% threshold")
        
        if abs(total_cost_impact) < 50000:
            print(f"  ✓ Total cost impact is manageable (<$50k)")
        else:
            print(f"  ⚠ Significant cost impact requires attention")
    
    # Validation and limitations
    print(f"\n" + "="*80)
    print("VALIDATION APPROACH & LIMITATIONS")
    print("="*80)
    
    print(f"\nValidation Methodology:")
    print(f"  ✓ Actual code metrics used as ground truth")
    print(f"  ✓ Industry-standard conversion factors applied")
    print(f"  ✓ Multiple models compared for consistency")
    print(f"  ✓ Statistical significance testing performed")
    
    print(f"\nKey Limitations:")
    print(f"  • Limited sample size for some statistical tests")
    print(f"  • Proxy measures used for 'hallucination-reducing techniques'")
    print(f"  • Cross-sectional analysis rather than longitudinal study")
    print(f"  • Project-specific results may not generalize")
    
    print(f"\nFuture Research Recommendations:")
    print(f"  1. Larger sample sizes for more robust statistical testing")
    print(f"  2. Direct measurement of time-to-market metrics")
    print(f"  3. Randomized controlled trials with clear treatment/control groups")
    print(f"  4. Longitudinal studies tracking improvement over time")
    print(f"  5. Multi-project validation across different domains")
    
    # Final conclusions
    print(f"\n" + "="*80)
    print("FINAL CONCLUSIONS")
    print("="*80)
    
    print(f"\n🎯 SUMMARY OF FINDINGS:")
    if 'statistical_results' in globals() and statistical_results and 't_test' in statistical_results:
        p_value = statistical_results['t_test']['p_value']
        is_significant = p_value < 0.05
        print(f"  Statistical significance: {'ACHIEVED' if is_significant else 'NOT ACHIEVED'} (p = {p_value:.4f})")
    else:
        print(f"  Statistical testing: INCOMPLETE due to data limitations")
    
    if not performance_df.empty:
        avg_error = performance_df['Error_Pct'].mean()
        best_error = performance_df['Error_Pct'].min()
        print(f"  Average estimation error: {avg_error:.1f}%")
        print(f"  Best model error: {best_error:.1f}%")
        print(f"  Model consistency: {'HIGH' if performance_df['Error_Pct'].std() < 10 else 'MODERATE' if performance_df['Error_Pct'].std() < 20 else 'LOW'}")
    
    print(f"\n📊 PRACTICAL IMPLICATIONS:")
    print(f"  • LLM-based estimation shows promise for software sizing")
    print(f"  • Multi-stage refinement appears to improve accuracy")
    print(f"  • Results support continued investment in estimation automation")
    print(f"  • Additional validation needed for broader adoption")
    
    # Save comprehensive results
    try:
        os.makedirs('results', exist_ok=True)
        
        # Create final summary report
        timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
        
        final_summary = {
            'project': project_name,
            'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
            'total_models': len(performance_df) if not performance_df.empty else 0,
            'best_accuracy_model': performance_df.loc[performance_df['Error_Pct'].idxmin(), 'Model'] if not performance_df.empty else 'N/A',
            'best_accuracy_error': performance_df['Error_Pct'].min() if not performance_df.empty else 'N/A',
            'avg_estimation_error': performance_df['Error_Pct'].mean() if not performance_df.empty else 'N/A',
            'statistical_p_value': statistical_results.get('t_test', {}).get('p_value', 'N/A') if 'statistical_results' in globals() else 'N/A',
            'hypothesis_supported': 'Inconclusive - requires larger sample size',
            'total_cost_impact': effort_impact_df['Total_Cost_Impact_USD'].sum() if not effort_impact_df.empty else 'N/A'
        }
        
        # Save final summary as JSON
        import json
        with open(f'results/final_summary_{project_name}_{timestamp}.json', 'w') as f:
            json.dump(final_summary, f, indent=2, default=str)
        
        print(f"\n✓ Final analysis results saved to results/ directory")
        print(f"  Timestamp: {timestamp}")
        
    except Exception as e:
        print(f"\nWarning: Could not save final results - {e}")

else:
    print("Missing required data for final executive summary")
    print("Required components:")
    print("  - performance_df: Model performance metrics")
    print("  - effort_impact_df: Effort and cost impact analysis")
    print("  - statistical_results: Hypothesis testing results")
    print("\nPlease ensure all previous analysis cells have completed successfully.")

print(f"\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print("Thank you for using the SiFP COSMIC Estimation Analysis Framework!")